In [1]:
import os, time
from vllm import LLM

/opt/app-root/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 04-03 17:12:01 [__init__.py:256] Automatically detected platform cuda.


2025-04-03 17:12:01,444	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
%run ../utilities/curl_utilities.ipynb
%run ../utilities/postgres_utilities.ipynb

In [3]:
model_name = "ibm-granite/granite-embedding-278m-multilingual"
number_data_rows = 40000

In [4]:
connect_db()

In [5]:
llm = LLM(model=model_name, task="embed")

INFO 04-03 17:12:02 [config.py:540] Found sentence-transformers tokenize configuration.
INFO 04-03 17:12:10 [config.py:436] Found sentence-transformers modules configuration.
INFO 04-03 17:12:10 [config.py:456] Found pooling configuration.
WARNING 04-03 17:12:10 [arg_utils.py:1765] --task embed is not supported by the V1 Engine. Falling back to V0. 
INFO 04-03 17:12:10 [llm_engine.py:241] Initializing a V0 LLM engine (v0.8.1) with config: model='ibm-granite/granite-embedding-278m-multilingual', speculative_config=None, tokenizer='ibm-granite/granite-embedding-278m-multilingual', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=512, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=Deco

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00, 105.53it/s]


INFO 04-03 17:12:13 [loader.py:429] Loading weights took 0.12 seconds
INFO 04-03 17:12:14 [model_runner.py:1146] Model loading took 0.5234 GB and 0.599682 seconds


In [6]:
sql = f"select productdisplayname from products where productdisplayname is not null limit {number_data_rows}"
results = execute_sql_results(sql)

In [7]:
test_description = "Starting batch embedding test using local vllm"
print(test_description)
print(len(test_description) * "-") 
tic = time.perf_counter()
print("Embeddings computed ...", end=" ")
descriptions = list()
for index, product in enumerate(results):
    descriptions.append(product[0])

batch_size = 40000
start_offset = 0
end_offset = batch_size
embeddings = list()
while start_offset < number_data_rows:
    print(start_offset, end_offset, end=" ")
    embeddings = embeddings + llm.embed(descriptions[start_offset:end_offset])
    start_offset = end_offset
    end_offset = end_offset + batch_size
    if end_offset > number_data_rows:
        end_offset = number_data_rows

toc = time.perf_counter()

assert len(embeddings) == number_data_rows
for e in embeddings:
    assert(len(e.outputs.embedding) == 768)

print("")
print(f"Time to compute {number_data_rows} embeddings: {toc - tic:0.4f} seconds")
print(len(test_description) * "-") 

Starting batch embedding test using local vllm
----------------------------------------------
Embeddings computed ... 0 40000 

Processed prompts: 100%|██████████| 40000/40000 [00:14<00:00, 2693.05it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]



Time to compute 40000 embeddings: 26.0498 seconds
----------------------------------------------


In [8]:
close_db()